In [3]:
import json
# read api key from tally_api_key.json
TALLY_API_KEY = json.load(open('tally_api_key.json'))['key']

In [35]:
import requests

url = "https://api.tally.xyz/query"

headers = {
  'Api-Key': TALLY_API_KEY,
  'Content-Type': 'application/json'
}

In [37]:
payload="{\"query\":\"query Chains {\\n    chains {\\n        id\\n    }\\n}\",\"variables\":{}}"

chains_response = requests.request("POST", url, headers=headers, data=payload)

chains_data = chains_response.json()
chains_data

{'data': {'chains': [{'id': 'eip155:412346'},
   {'id': 'eip155:42'},
   {'id': 'eip155:80001'},
   {'id': 'eip155:137'},
   {'id': 'eip155:42161'},
   {'id': 'eip155:421611'},
   {'id': 'eip155:43113'},
   {'id': 'eip155:5'},
   {'id': 'eip155:534353'},
   {'id': 'eip155:1'},
   {'id': 'eip155:100'},
   {'id': 'eip155:10'},
   {'id': 'eip155:4'},
   {'id': 'eip155:56'},
   {'id': 'eip155:42170'},
   {'id': 'eip155:97'},
   {'id': 'eip155:43114'},
   {'id': 'eip155:84531'},
   {'id': 'eip155:420'},
   {'id': 'eip155:1284'},
   {'id': 'eip155:69'},
   {'id': 'eip155:11155111'},
   {'id': 'eip155:421613'}]}}

In [39]:
chains = chains_data['data']['chains']
chains = [c['id'] for c in chains]
chains

['eip155:412346',
 'eip155:42',
 'eip155:80001',
 'eip155:137',
 'eip155:42161',
 'eip155:421611',
 'eip155:43113',
 'eip155:5',
 'eip155:534353',
 'eip155:1',
 'eip155:100',
 'eip155:10',
 'eip155:4',
 'eip155:56',
 'eip155:42170',
 'eip155:97',
 'eip155:43114',
 'eip155:84531',
 'eip155:420',
 'eip155:1284',
 'eip155:69',
 'eip155:11155111',
 'eip155:421613']

In [40]:
def make_proposals_query(chain: str) -> str:
    query = """query Proposals {
        proposals(
            chainId: "%s"
        ) {
            platform_deployment_id: governanceId
            proposal_id: id
            author: proposer {
                id
            }
            date: start {
                timestamp
            }
            voteStats {
                support
                votes
            }
        }
    }"""
    return query % chain

In [45]:
import time

responses_by_chain = []
for chain in chains:
    status_code = 999
    while status_code != 200:
        query = make_proposals_query(chain)
        response = requests.request("POST", url, headers=headers, data=json.dumps({'query': query}))
        status_code = response.status_code
        if status_code != 200:
            print('error', chain, response.status_code)
            # wait
            time.sleep(5)
    responses_by_chain.append(response.json())
    print('done', chain, response.status_code)
    time.sleep(2)

done eip155:412346 200
done eip155:42 200
done eip155:80001 200
done eip155:137 200
done eip155:42161 200
done eip155:421611 200
done eip155:43113 200
done eip155:5 200
done eip155:534353 200
done eip155:1 200
done eip155:100 200
done eip155:10 200
done eip155:4 200
done eip155:56 200
done eip155:42170 200
done eip155:97 200
done eip155:43114 200
done eip155:84531 200
done eip155:420 200
done eip155:1284 200
done eip155:69 200
done eip155:11155111 200
done eip155:421613 200


In [46]:
import pandas as pd
# make a dataframe of all proposals
proposals = []
for response in responses_by_chain:
    proposals += response['data']['proposals']
df = pd.DataFrame(proposals)
df

,platform_deployment_id,proposal_id,author,date,voteStats
0,eip155:42:0x6975BEA5FCB19d108bEc5040C36579E34C...,1060591009403591849631122936066727758328463284...,{'id': 'eip155:42:0x61A3d01D4F7Ab602E44F24E7D1...,{'timestamp': '2022-03-12T09:14:49Z'},"[{'support': 'FOR', 'votes': '0'}, {'support':..."
1,eip155:42:0x4368187326E5319e7fA8eEC719E76928F3...,6369967960540701657553784708872735735172925018...,{'id': 'eip155:42:0x81AFd6d5a43f20B570030f355b...,{'timestamp': '2022-03-22T23:05:53Z'},"[{'support': 'FOR', 'votes': '1'}, {'support':..."
2,eip155:80001:0x30E6f95cdF35d85857b38bD7bE4073A...,1514232309894785002504531422483056691445042232...,{'id': 'eip155:80001:0xC9E6e248928aC18eD6b1036...,{'timestamp': '2022-11-09T01:19:00Z'},"[{'support': 'FOR', 'votes': '0'}, {'support':..."
3,eip155:80001:0x8186385eE73E140496aE83C125d0Cc2...,1114923375315352498572725689646100447453417963...,{'id': 'eip155:80001:0x5ABD73cE8Aec6fc8e2450e4...,{'timestamp': '2022-08-16T07:26:00Z'},"[{'support': 'FOR', 'votes': '1'}, {'support':..."
4,eip155:80001:0x8186385eE73E140496aE83C125d0Cc2...,2082617619439618836836527330976067803780310954...,{'id': 'eip155:80001:0x5ABD73cE8Aec6fc8e2450e4...,{'timestamp': '2022-08-17T06:39:38Z'},"[{'support': 'FOR', 'votes': '1'}, {'support':..."
...,...,...,...,...,...
7792,eip155:421613:0x7C764b23e87296202963206584EF16...,1917071857544985608051387556539652800945726731...,{'id': 'eip155:421613:0x3382Bb7214c109f12Ffe8a...,{'timestamp': '2023-03-28T17:56:24Z'},"[{'support': 'FOR', 'votes': '1'}, {'support':..."
7793,eip155:421613:0x7C764b23e87296202963206584EF16...,1030333202873856643768323554682388205845820513...,{'id': 'eip155:421613:0x3382Bb7214c109f12Ffe8a...,{'timestamp': '2023-03-29T02:04:24Z'},"[{'support': 'FOR', 'votes': '0'}, {'support':..."
7794,eip155:421613:0x7C764b23e87296202963206584EF16...,4646408104848257255297190757734404882319472750...,{'id': 'eip155:421613:0x3382Bb7214c109f12Ffe8a...,{'timestamp': '2023-03-29T02:20:24Z'},"[{'support': 'FOR', 'votes': '1'}, {'support':..."
7795,eip155:421613:0x17BccCC8E7c0DC62453a508988b618...,1190204112804234920253299990098496707197280664...,{'id': 'eip155:421613:0xD8a394e7d7894bDF2C5713...,{'timestamp': '2023-06-11T15:45:24Z'},"[{'support': 'FOR', 'votes': '1'}, {'support':..."


In [47]:
# replace author column with author_id
df['author'] = df['author'].apply(lambda x: x['id'])
df.head()

,platform_deployment_id,proposal_id,author,date,voteStats
0,eip155:42:0x6975BEA5FCB19d108bEc5040C36579E34C...,1060591009403591849631122936066727758328463284...,eip155:42:0x61A3d01D4F7Ab602E44F24E7D139395c67...,{'timestamp': '2022-03-12T09:14:49Z'},"[{'support': 'FOR', 'votes': '0'}, {'support':..."
1,eip155:42:0x4368187326E5319e7fA8eEC719E76928F3...,6369967960540701657553784708872735735172925018...,eip155:42:0x81AFd6d5a43f20B570030f355b58bF3172...,{'timestamp': '2022-03-22T23:05:53Z'},"[{'support': 'FOR', 'votes': '1'}, {'support':..."
2,eip155:80001:0x30E6f95cdF35d85857b38bD7bE4073A...,1514232309894785002504531422483056691445042232...,eip155:80001:0xC9E6e248928aC18eD6b103653cBcF66...,{'timestamp': '2022-11-09T01:19:00Z'},"[{'support': 'FOR', 'votes': '0'}, {'support':..."
3,eip155:80001:0x8186385eE73E140496aE83C125d0Cc2...,1114923375315352498572725689646100447453417963...,eip155:80001:0x5ABD73cE8Aec6fc8e2450e4f4c5FAc7...,{'timestamp': '2022-08-16T07:26:00Z'},"[{'support': 'FOR', 'votes': '1'}, {'support':..."
4,eip155:80001:0x8186385eE73E140496aE83C125d0Cc2...,2082617619439618836836527330976067803780310954...,eip155:80001:0x5ABD73cE8Aec6fc8e2450e4f4c5FAc7...,{'timestamp': '2022-08-17T06:39:38Z'},"[{'support': 'FOR', 'votes': '1'}, {'support':..."


In [48]:
# read date.timestamp as date
df['date'] = df['date'].apply(lambda x: x['timestamp'])
# convert date to datetime from 2022-09-12T06:49:26Z format
df['date'] = pd.to_datetime(df['date'])
df.head()

,platform_deployment_id,proposal_id,author,date,voteStats
0,eip155:42:0x6975BEA5FCB19d108bEc5040C36579E34C...,1060591009403591849631122936066727758328463284...,eip155:42:0x61A3d01D4F7Ab602E44F24E7D139395c67...,2022-03-12 09:14:49+00:00,"[{'support': 'FOR', 'votes': '0'}, {'support':..."
1,eip155:42:0x4368187326E5319e7fA8eEC719E76928F3...,6369967960540701657553784708872735735172925018...,eip155:42:0x81AFd6d5a43f20B570030f355b58bF3172...,2022-03-22 23:05:53+00:00,"[{'support': 'FOR', 'votes': '1'}, {'support':..."
2,eip155:80001:0x30E6f95cdF35d85857b38bD7bE4073A...,1514232309894785002504531422483056691445042232...,eip155:80001:0xC9E6e248928aC18eD6b103653cBcF66...,2022-11-09 01:19:00+00:00,"[{'support': 'FOR', 'votes': '0'}, {'support':..."
3,eip155:80001:0x8186385eE73E140496aE83C125d0Cc2...,1114923375315352498572725689646100447453417963...,eip155:80001:0x5ABD73cE8Aec6fc8e2450e4f4c5FAc7...,2022-08-16 07:26:00+00:00,"[{'support': 'FOR', 'votes': '1'}, {'support':..."
4,eip155:80001:0x8186385eE73E140496aE83C125d0Cc2...,2082617619439618836836527330976067803780310954...,eip155:80001:0x5ABD73cE8Aec6fc8e2450e4f4c5FAc7...,2022-08-17 06:39:38+00:00,"[{'support': 'FOR', 'votes': '1'}, {'support':..."


In [49]:
# remove tz info from date
df['date'] = df['date'].apply(lambda x: x.replace(tzinfo=None))
df.head()

,platform_deployment_id,proposal_id,author,date,voteStats
0,eip155:42:0x6975BEA5FCB19d108bEc5040C36579E34C...,1060591009403591849631122936066727758328463284...,eip155:42:0x61A3d01D4F7Ab602E44F24E7D139395c67...,2022-03-12 09:14:49,"[{'support': 'FOR', 'votes': '0'}, {'support':..."
1,eip155:42:0x4368187326E5319e7fA8eEC719E76928F3...,6369967960540701657553784708872735735172925018...,eip155:42:0x81AFd6d5a43f20B570030f355b58bF3172...,2022-03-22 23:05:53,"[{'support': 'FOR', 'votes': '1'}, {'support':..."
2,eip155:80001:0x30E6f95cdF35d85857b38bD7bE4073A...,1514232309894785002504531422483056691445042232...,eip155:80001:0xC9E6e248928aC18eD6b103653cBcF66...,2022-11-09 01:19:00,"[{'support': 'FOR', 'votes': '0'}, {'support':..."
3,eip155:80001:0x8186385eE73E140496aE83C125d0Cc2...,1114923375315352498572725689646100447453417963...,eip155:80001:0x5ABD73cE8Aec6fc8e2450e4f4c5FAc7...,2022-08-16 07:26:00,"[{'support': 'FOR', 'votes': '1'}, {'support':..."
4,eip155:80001:0x8186385eE73E140496aE83C125d0Cc2...,2082617619439618836836527330976067803780310954...,eip155:80001:0x5ABD73cE8Aec6fc8e2450e4f4c5FAc7...,2022-08-17 06:39:38,"[{'support': 'FOR', 'votes': '1'}, {'support':..."


In [50]:
# sum the 'votes' value for each item in voteStats and save as votes_count
df['votes_count'] = df['voteStats'].apply(lambda x: sum([int(i['votes']) for i in x]))
# drop voteStats column
df.drop('voteStats', axis=1, inplace=True)
df.head()

,platform_deployment_id,proposal_id,author,date,votes_count
0,eip155:42:0x6975BEA5FCB19d108bEc5040C36579E34C...,1060591009403591849631122936066727758328463284...,eip155:42:0x61A3d01D4F7Ab602E44F24E7D139395c67...,2022-03-12 09:14:49,0
1,eip155:42:0x4368187326E5319e7fA8eEC719E76928F3...,6369967960540701657553784708872735735172925018...,eip155:42:0x81AFd6d5a43f20B570030f355b58bF3172...,2022-03-22 23:05:53,3
2,eip155:80001:0x30E6f95cdF35d85857b38bD7bE4073A...,1514232309894785002504531422483056691445042232...,eip155:80001:0xC9E6e248928aC18eD6b103653cBcF66...,2022-11-09 01:19:00,0
3,eip155:80001:0x8186385eE73E140496aE83C125d0Cc2...,1114923375315352498572725689646100447453417963...,eip155:80001:0x5ABD73cE8Aec6fc8e2450e4f4c5FAc7...,2022-08-16 07:26:00,1
4,eip155:80001:0x8186385eE73E140496aE83C125d0Cc2...,2082617619439618836836527330976067803780310954...,eip155:80001:0x5ABD73cE8Aec6fc8e2450e4f4c5FAc7...,2022-08-17 06:39:38,3


In [51]:
df['platform'] = 'tally'
# reorder cols so that platform is first
df = df[['platform', 'platform_deployment_id', 'proposal_id', 'author', 'date', 'votes_count']]
df.head()

,platform,platform_deployment_id,proposal_id,author,date,votes_count
0,tally,eip155:42:0x6975BEA5FCB19d108bEc5040C36579E34C...,1060591009403591849631122936066727758328463284...,eip155:42:0x61A3d01D4F7Ab602E44F24E7D139395c67...,2022-03-12 09:14:49,0
1,tally,eip155:42:0x4368187326E5319e7fA8eEC719E76928F3...,6369967960540701657553784708872735735172925018...,eip155:42:0x81AFd6d5a43f20B570030f355b58bF3172...,2022-03-22 23:05:53,3
2,tally,eip155:80001:0x30E6f95cdF35d85857b38bD7bE4073A...,1514232309894785002504531422483056691445042232...,eip155:80001:0xC9E6e248928aC18eD6b103653cBcF66...,2022-11-09 01:19:00,0
3,tally,eip155:80001:0x8186385eE73E140496aE83C125d0Cc2...,1114923375315352498572725689646100447453417963...,eip155:80001:0x5ABD73cE8Aec6fc8e2450e4f4c5FAc7...,2022-08-16 07:26:00,1
4,tally,eip155:80001:0x8186385eE73E140496aE83C125d0Cc2...,2082617619439618836836527330976067803780310954...,eip155:80001:0x5ABD73cE8Aec6fc8e2450e4f4c5FAc7...,2022-08-17 06:39:38,3


In [ ]:
# save as csv
df.to_csv('tally_proposals.csv', index=False)